In [1]:
def dmi(df1, period):
    df = df1[['High', 'Low', 'Open', 'Close', 'Volume']].copy()
    df['+DM'] = abs(df['High'] - df['High'].shift(1))
    df['-DM'] = abs(df['Low'].shift(1) - df['Low'])
    df['TR'] = np.nan
    df['smooth_TR'] = np.nan
    df['ADX'] = np.nan

    for i in range(len(df['Close'])):
        df.loc[[df.index[i]], ['TR']] = np.max([ df['High'][i] - df['Low'][i], 
                                   abs(df['High'][i] - df['Close'][i-1]), 
                                   abs(df['Low'][i] - df['Close'][i-1]) ])
    df.loc[[df.index[0]], ['TR']] = np.nan

    df.loc[df.index[period], ['smooth_TR']] = 0
    for i in range(1, period+1):
        df.loc[df.index[period], 'smooth_TR'] += df['TR'][i]
    for i in range(period+1, len(df)):
        df.loc[df.index[i], 'smooth_TR'] = df['smooth_TR'][i-1] - (df['smooth_TR'][i-1] / period) + df['TR'][i]
        
    df['+DI'] = np.nan
    df['-DI'] = np.nan
    df['smooth_+DMI'] = np.nan
    df['smooth_-DMI'] = np.nan

    for i in range(len(df)):
        if df['+DM'][i] < 0:
            df.loc[df.index[i], ['+DI']] = 0
        elif df['+DM'][i] > df['-DM'][i]:
            df.loc[df.index[i], ['+DI']] = df['+DM'][i]
        else:
            df.loc[df.index[i], ['+DI']] = 0

        if df['-DM'][i] < 0:
            df.loc[df.index[i], ['-DI']] = 0
        elif df['-DM'][i] > df['+DM'][i]:
            df.loc[df.index[i], ['-DI']] = df['-DM'][i]
        else:
            df.loc[df.index[i], ['-DI']] = 0

    df.loc[df.index[period], ['smooth_+DMI']] = 0
    for i in range(1, period+1):
        df.loc[df.index[period], 'smooth_+DMI'] += df['+DI'][i]
    for i in range(period+1, len(df)):
        df.loc[df.index[i], 'smooth_+DMI'] = df['smooth_+DMI'][i-1] - (df['smooth_+DMI'][i-1] / period) + df['+DI'][i]

    df.loc[df.index[period], ['smooth_-DMI']] = 0
    for i in range(1, period+1):
        df.loc[df.index[period], 'smooth_-DMI'] += df['-DI'][i]
    for i in range(period+1, len(df)):
        df.loc[df.index[i], 'smooth_-DMI'] = df['smooth_-DMI'][i-1] - (df['smooth_-DMI'][i-1] / period) + df['-DI'][i]
        
    df['DX'] = abs(df['smooth_+DMI'] - df['smooth_-DMI']) / (df['smooth_+DMI'] + df['smooth_-DMI']) * 100

    df.loc[df.index[2*period-1], ['ADX']] = 0
    for i in range(period, 2*period):
        df.loc[df.index[2*period-1], 'ADX'] += df.loc[df.index[i], 'DX']
    df.loc[df.index[2*period-1], ['ADX']] = df['ADX'][2*period-1] / period
    for i in range(2*period, len(df)):
        df.loc[df.index[i], 'ADX'] = (df['ADX'][i-1] * (period-1) + df['DX'][i]) / period

    #df['ADXR'] = df['ADX'].rolling(window=period+1).mean()
    
    return(df[['smooth_+DMI', 'smooth_-DMI', 'ADX']])

df['smooth_+DM'] = np.nan
df['smooth_-DM'] = np.nan

df.loc[df.index[period], ['smooth_+DM']] = 0
for i in range(1, period+1):
    df.loc[df.index[period], 'smooth_+DM'] += df['+DM'][i]
for i in range(period+1, len(df)):
    df.loc[df.index[i], 'smooth_+DM'] = df['smooth_+DM'][i-1] - (df['smooth_+DM'][i-1] / period) + df['+DM'][i]
    
df.loc[df.index[period], ['smooth_-DM']] = 0
for i in range(1, period+1):
    df.loc[df.index[period], 'smooth_-DM'] += df['-DM'][i]
for i in range(period+1, len(df)):
    df.loc[df.index[i], 'smooth_-DM'] = df['smooth_-DM'][i-1] - (df['smooth_-DM'][i-1] / period) + df['-DM'][i]
df['smooth_+DI'] = df['smooth_+DM'] / df['smooth_TR'] * 100
df['smooth_-DI'] = df['smooth_-DM'] / df['smooth_TR'] * 100